In [1]:
import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM
import os
import re
import sys
from typing import List
import json

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)

import torch
from datasets import load_dataset
import pandas as pd
 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams

# Update this path to wherever tensorboard is installed in your environment
os.environ['TENSORBOARD_BINARY'] = '/opt/conda/envs/py310/bin/tensorboard'


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /opt/conda did not contain libcudart.so as expected! Searching further paths...
  warn(msg)


In [3]:
%matplotlib inline
sns.set(rc={'figure.figsize':(10, 7)})
sns.set(rc={'figure.dpi':100})
sns.set(style='white', palette='muted', font_scale=1.2)
 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [4]:
BASE_MODEL = "eachadea/vicuna-13b-1.1"

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
data = load_dataset("json", data_files="../data/medical_prompts.json")
data["train"]

Found cached dataset json (/home/jupyter/.cache/huggingface/datasets/json/default-74cd44681a8bfc82/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 19577
})

In [6]:
CUTOFF_LEN = 256

def generate_prompt(data_point):
    return f"""Below is an instruction that provides a context for the paired input that asks a question. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""

def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
 
    result["labels"] = result["input_ids"].copy()
 
    return result
 
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

In [7]:
train_val = data["train"].train_test_split(
    test_size=500, shuffle=True, seed=42
)
train_data = (
    train_val["train"].map(generate_and_tokenize_prompt)
)
val_data = (
    train_val["test"].map(generate_and_tokenize_prompt)
)

Loading cached split indices for dataset at /home/jupyter/.cache/huggingface/datasets/json/default-74cd44681a8bfc82/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-21dee988d6256f93.arrow and /home/jupyter/.cache/huggingface/datasets/json/default-74cd44681a8bfc82/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-d318d8ce134a9bc8.arrow
Loading cached processed dataset at /home/jupyter/.cache/huggingface/datasets/json/default-74cd44681a8bfc82/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-34cf068b5fbd02c2.arrow
Loading cached processed dataset at /home/jupyter/.cache/huggingface/datasets/json/default-74cd44681a8bfc82/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-3270482d00cce608.arrow


In [8]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

BATCH_SIZE = 256
MICRO_BATCH_SIZE = 8
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 300
OUTPUT_DIR = "experiments"  # Path to where your training model will be saved

In [9]:
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 6553600 || all params: 13022417920 || trainable%: 0.05032552357220002


In [10]:
training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=100,
    max_steps=TRAIN_STEPS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=5,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=5,
    save_steps=10,
    output_dir=OUTPUT_DIR,
    save_total_limit=3,
    load_best_model_at_end=True,
    report_to="tensorboard"
)

In [11]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

In [12]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model.config.use_cache = False
old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))
 
model = torch.compile(model)

In [13]:
trainer.train()

Step,Training Loss,Validation Loss
5,2.504800,2.493454
10,2.501200,2.471636
15,2.465900,2.414007
20,2.389400,2.309166
25,2.250300,2.150407
30,2.074900,1.957954
35,1.876700,1.761527
40,1.663200,1.569331
45,1.475500,1.422467
50,1.390900,1.384198


KeyboardInterrupt: 

In [14]:
model.save_pretrained(OUTPUT_DIR)

In [17]:
from huggingface_hub import notebook_login
 
notebook_login()

In [18]:
model.push_to_hub("kmnis/DocScribe", use_auth_token=True)

CommitInfo(commit_url='https://huggingface.co/kmnis/medVicuna/commit/dd0cae86dbe50dba43f772ae55d8faae0fc69a83', commit_message='Upload model', commit_description='', oid='dd0cae86dbe50dba43f772ae55d8faae0fc69a83', pr_url=None, pr_revision=None, pr_num=None)

In [2]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs